# shubnikov de haas oscillations analysis




#### python libraries and notes
* https://plotly.com/python/

* https://github.com/fbeilstein/machine_learning


## reading and pre processing data

We start with importing data from csv. Data was saved as **(field,T ; voltave, V)**.

Function **readSdh** reads from the csv and creates useful columns. 

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#plotting libraries
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots 

import scipy as sp
import scipy.fftpack




def readSdH(csvName, current): #function to read and preprocess sdH oscillations 
    df = pd.read_csv(csvName, header=None)
    df.columns = ['field', 'voltage']

    df['resistance']= df['voltage']/current
    df['1/B'] = 1/df['field']

    return df

data4K =readSdH('data/sdH4K.csv',50* 10**(-9))
data10K =readSdH('data/sdH10K.csv',100* 10**(-9))

#defining columns for relative resistance oscillations and 1/B field
data4K['dR/R'] = (data4K['resistance']-data4K['resistance'][0])/data4K['resistance'][0]
data10K['dR/R'] = (data10K['resistance']-data10K['resistance'][0])/data10K['resistance'][0]


### plotting the oscillations

In [5]:
def plotSdH(df): 
    fig = px.scatter(df, x = df['field'], y = df['dR/R'], width = 600, height = 400, template = 'simple_white')
    fig.update_layout(
    xaxis_title=r'Magnetic field, T',
    yaxis_title=r'$\frac{R_i-R(0)}{R(0)}$', font=dict(
        size=15
    ))
#markers style
    fig.update_traces(marker=dict(size=3,opacity=0.8, color = "#7AC5CD",
                              line=dict(width=1,
                                        color="#53868B")),
                  selector=dict(mode='markers'))
    fig.show()





fig = go.Figure()

fig.add_trace(go.Scatter(
    x = data4K['field'], y = data4K['dR/R'],
    name="4K"       # this sets its legend entry
))

fig.add_trace(go.Scatter(
    x = data10K['field'], y = data10K['dR/R'],
    name="10K"
))


fig.update_layout(height=800, width=1000,template = 'simple_white', title_text="Stacked Subplots")
fig.update_layout(
    title=r'Mesa sdH oscillations',
    xaxis_title=r'Magnetic field, T',
    yaxis_title=r'$\frac{R_i-R(0)}{R(0)}$', font=dict(
        size=20
    )
)
fig.update_layout(legend_orientation="h")

fig.show()




## Filtering data

as we are mainly interested in oscillationg signal, we can pass all of our data through a high pass filter to get rid of static background.  


- https://www.youtube.com/watch?v=s2K1JfNR7Sc&t=339s
- https://www.youtube.com/watch?v=wJgBB4EsUHw&t=36s
- https://www.youtube.com/watch?v=dmzikG1jZpU

In [ ]:
def highpass_filter(y, sr):
  filter_stop_freq = 70  # Hz
  filter_pass_freq = 100  # Hz
  filter_order = 1001

  # High-pass filter
  nyquist_rate = sr / 2.
  desired = (0, 0, 1, 1)
  bands = (0, filter_stop_freq, filter_pass_freq, nyquist_rate)
  filter_coefs = signal.firls(filter_order, bands, desired, nyq=nyquist_rate)

  # Apply high-pass filter
  filtered_audio = signal.filtfilt(filter_coefs, [1], y)
  return filtered_audio

## Getting $k_\text{F}$ from SdH oscillations

Here we define $f$ as the frequency of oscillations over $1/B$. 

The frequencies of the quantum oscillations of resistance $f$ are related to the cross section of the Fermi surfaces $A_{\text{k}}$ in the $k_{\parallel}$ plane by the Onsager relation:

$f = \frac{h}{4 \pi^2 e} A_{\text{k}} = \frac{h}{4 \pi e} k_{\parallel} ^2$


*Note*: we use almost free electron model, therefore, we consider Fermi surface to be a sphere. The section of the sphere is a circle. That means that $A_{\text{k}} = \pi k^2$.


First, we are getting the frequency by performing FFT on resistance oscillations over $1/B$


In [3]:
#plot inputs 


def plot1OverB(df, insetA, insetB):

    df = df.loc[df['1/B'] <.35] 
    insetDF = df.loc[(df['1/B'] >= insetA) & (df['1/B'] <= insetB)]
    
    
   
    
    fig = px.scatter(data4K, x = df['1/B'], y = df['dR/R'], width = 1000, height = 600, template = 'simple_white')
    fig.update_layout(
        xaxis_title=r'1/B, 1/T',
        yaxis_title=r'$\frac{R_i-R(0)}{R(0)}$', font=dict(
        size=16
    ))
    #markers style
    fig.update_traces(marker=dict(size=3,opacity=1, color = "black",
                              line=dict(width=.5,
                                        color="#53868B")),
                  selector=dict(mode='markers'))
    fig.show()
    
plot1OverB(data4K,1,1)
print('Mesa 4K')
plot1OverB(data10K,1,1)
print('Mesa 10K')

Mesa 4K


Mesa 10K


### non uniform FFT

the issue with our data is that it is not evenly spaced in terms of 1/B (because the sweep is spaced in terms of B). I didn't want to work with the library so I am trying to write a function for 'irregular fourier transform'that just accounts for the time stamps. (https://mathematica.stackexchange.com/questions/110064/how-to-realize-a-fourier-transform-on-a-non-uniform-sampling-data this example worked in mathemtica).

here we define fft as the following:

$X(f_k)=\frac{\sum_{n=0}^{N-1} x_{n} e^{-2 \pi i t_{n} f_{k}}}{N}, \quad 0 \leq k \leq M$


unfortunately, right now it is VERY slow. need to fix that ASAP. There are different non uniform Fast Fourier Transform algorithms. Need to take a look


#### useful tutorials 
- what Fourier transform is https://www.youtube.com/watch?v=spUNpyF58BY
- what fast fourier transform is https://www.youtube.com/watch?v=E8HeD-MUrjY


In [30]:
#input for a function -- (dataset_Name, starting_freq, end_freq, step (be careful, changes computation speed dramatically))

def ift(df, f1, f2, step):
    dfInt = df.loc[df['1/B'] <.5] 
    n = dfInt.shape[0] #number of points in 'time'
    freq = np.arange(f1, f2, step) #create a set of frequencies
    vals = dfInt['dR/R'].values #values in our fft transform
    times = dfInt['1/B'].values #'times' in our fft transform
    transform = np.zeros(len(freq),dtype=complex)
    #df['freq'] = freq
    df.loc[:,'freq'] = pd.Series(freq)  #this helps avoiding creating additional data structure. however, the rows don't make much sense anymore. unlucky :(
    for i in range(len(freq)):
        transform[i] = sum(vals[k]*np.exp(-2j*times[k]*np.pi*freq[i]) for k in range(n))/n
    
    df.loc[:,'transform'] = pd.Series(np.absolute(transform))
  
    fig = px.scatter(x=freq, y=np.absolute(transform),width = 1000, height = 800, template = 'simple_white')
    fig.update_layout(
    xaxis_title=r'Magnetic field, T',
    yaxis_title=r'Amplitude', font=dict(
        size=18
    ))
#markers style
    fig.update_traces(marker=dict(size=2,opacity=1, color = "black",
                              line=dict(width=.5,
                                        color="#53868B")),
                  selector=dict(mode='markers'))
    fig.show()


    

In [31]:
dft4K = ift(data4K,1,300, 0.1)
print('dft 4K mesa')


dft 4K mesa


In [32]:
dft10K = ift(data10K,1,300,.1)
print('dft 10K mesa')


dft 10K mesa


### extracting the frequency from the fft

In [61]:
#this returns the maximum frequency of the DFT
frequency4K = data4K.loc[data4K['transform'].idxmax()][5]
print('4K frequency is %f Hz' % frequency4K)

#this returns the maximum frequency of the DFT
frequency10K = data10K.loc[data10K['transform'].idxmax()][5]
print('10K frequency is %f Hz' % frequency10K)


import scipy.constants 
 
kf4K = np.sqrt(2*scipy.constants.e*frequency4K/scipy.constants.hbar)
print('4K kFermi is %.3E m^-2' %kf4K)

kf10K = np.sqrt(2*scipy.constants.e*frequency10K/scipy.constants.hbar)
print('10K kFermi is %.3E m^-2' %kf10K)


4K frequency is 128.000000 Hz
10K frequency is 127.800000 Hz
4K kFermi is 6.236E+08 m^-2
10K kFermi is 6.232E+08 m^-2


## Finding effective mass 

first we need to detect the peaks and get their coordinates

this thread explains why prominence is a good parameter for finding peaks
- https://stackoverflow.com/questions/1713335/peak-finding-algorithm-for-python-scipy

In [43]:
from scipy.signal import find_peaks

#watch out! the lower the prominence the slower the algorithm works
def peaks_and_valleys(df, prominence_val):
    
    data_x = df['field'].values
    data_y = df['dR/R'].values 
    indices = find_peaks(data_y, prominence = prominence_val)[0]
    valleys = find_peaks(-data_y, prominence = prominence_val)[0]

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=data_y,
        mode='lines+markers',
        name='Original Plot'
    ))

    fig.add_trace(go.Scatter(
        x=indices,
        y=[data_y[j] for j in indices],
        mode='markers',
        marker=dict(
            size=8,
            color='red',
            symbol='cross'
        ),
        name='Detected Peaks'
    ))


    fig.add_trace(go.Scatter(
        x=valleys,
        y=[data_y[j] for j in valleys],
        mode='markers',
        marker=dict(
            size=8,
            color='yellow',
            symbol='cross'
        ),
        name='Detected Valleys'
    ))


    fig.update_layout(
        width = 800, height = 600, template = 'simple_white',
        xaxis_title=r'Magnetic field, T',
        yaxis_title=r'Amplitude', font=dict(
            size=18
        ))


    fig.show()
    
peaks_and_valleys(data4K,.5)
peaks_and_valleys(data10K,.1)
